In [1]:
# Login
!az login > /dev/null

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code DUSM69KD7 to authenticate.


In [2]:
# Connect to workspace
from azureml.core import Workspace
ws = Workspace.from_config()

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [4]:
# Create Docker environment
from azureml.core import Environment

myenv = Environment(name="myenv")
myenv.docker.enabled = True
with open("Dockerfile", "r") as f:
    dockerfile=f.read()
myenv.docker.base_dockerfile = dockerfile
myenv.python.user_managed_dependencies = True

Property base_image is mutually exclusive with base_dockerfile. Reset base_image to None


In [6]:
# Register the Docker environment
myenv.register(workspace=ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": "# Base image\nFROM mcr.microsoft.com/azureml/base\n\n# Avoid warnings by switching to noninteractive\nENV DEBIAN_FRONTEND=noninteractive\n\n# Non-root user with sudo access\nARG USERNAME=default\nARG USER_UID=1000\nARG USER_GID=$USER_UID\n\n# Apt deps\nRUN apt-get update \\\n    && apt-get -y install --no-install-recommends apt-utils dialog 2>&1 \\\n    && apt-get -y install \\\n        sudo \\\n        curl \\\n    && apt-get autoremove -y \\\n    && apt-get clean -y \\\n    && rm -rf /var/lib/apt/lists/* \\\n    # az CLI\n    && curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash \\\n    && apt-get clean -y \\\n    && rm -rf /var/lib/apt/lists/* \\\n    # pip deps\n    && pip install azureml-sdk[notebooks,automl,explain] \\\n    # create non-ro

In [7]:
# List registered envs
from azureml.core import Environment

env_names = Environment.list(workspace=ws)
for env_name in env_names:
    print('Name:',env_name)

Name: myenv
Name: AzureML-AutoML
Name: AzureML-PyTorch-1.0-GPU
Name: AzureML-Scikit-learn-0.20.3
Name: AzureML-TensorFlow-1.12-CPU
Name: AzureML-PyTorch-1.2-GPU
Name: AzureML-TensorFlow-2.0-GPU
Name: AzureML-TensorFlow-2.0-CPU
Name: AzureML-Chainer-5.1.0-GPU
Name: AzureML-TensorFlow-1.13-CPU
Name: AzureML-Minimal
Name: AzureML-Chainer-5.1.0-CPU
Name: AzureML-PyTorch-1.4-GPU
Name: AzureML-PySpark-MmlSpark-0.15
Name: AzureML-PyTorch-1.3-CPU
Name: AzureML-PyTorch-1.1-GPU
Name: AzureML-TensorFlow-1.10-GPU
Name: AzureML-PyTorch-1.2-CPU
Name: AzureML-TensorFlow-1.13-GPU
Name: AzureML-TensorFlow-1.10-CPU
Name: AzureML-PyTorch-1.3-GPU
Name: AzureML-PyTorch-1.4-CPU
Name: AzureML-Tutorial
Name: AzureML-PyTorch-1.0-CPU
Name: AzureML-PyTorch-1.1-CPU
Name: AzureML-TensorFlow-1.12-GPU
Name: AzureML-Designer-VowpalWabbit
Name: AzureML-TensorFlow-2.2-GPU
Name: AzureML-TensorFlow-2.2-CPU
Name: AzureML-PyTorch-1.6-CPU
Name: AzureML-PyTorch-1.6-GPU
Name: AzureML-Triton
Name: AzureML-TensorFlow-2.3-CPU
Na

In [11]:
# Create a compute target
from azureml.core.compute import ComputeTarget, ComputeInstance

# Load the workspace from the saved config file
ws = Workspace.from_config()

# Specify a name for the compute (unique within the workspace)
compute_name = 'nossh'

# Define compute configuration
compute_config = ComputeInstance.provisioning_configuration(vm_size='STANDARD_DS11_V2')

# Create the compute
compute_instance = ComputeTarget.create(ws, compute_name, compute_config)
# compute_instance.wait_for_completion(show_output=True)

In [12]:
# Submit to target
from azureml.core import Experiment, ScriptRunConfig

# Create a script config
script_config = ScriptRunConfig(source_directory='scripts',
                                script='diabetes_lr_ds.py',
                                arguments = ['--ds-name', 'diabetes'],
                                compute_target=compute_instance,
                                environment=myenv)

# Submit the experiment
experiment = Experiment(workspace=ws, name='diabetes_lr')
run = experiment.submit(config=script_config)